↓ ↓ ↓ A Chronicle of Early Failures ↓ ↓ ↓

In [1]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install openpyxl

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,089 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,889 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [53.5 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.5 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,390 kB]
Hit:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:13 https://ppa.launchpadconten

In [18]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
from openpyxl.styles import Alignment, Font, PatternFill
import time

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument('--window-size=1920x1080')

# Initialize the WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Function to scroll and collect comments
def scroll_and_collect_comments(driver, video_url):

    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait for new comments to load
    time.sleep(5)

    # Extract comments
    comment_elements = driver.find_elements(By.CSS_SELECTOR, "#content-text")
    comments = np.array([element.text for element in comment_elements])  # Convert comments to NumPy array
    comments = np.unique(comments)

    return comments

# Function to navigate to a YouTube video and extract comments
def get_youtube_comments(video_url):
    driver.get(video_url)

    # Wait for the page to load completely
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(5)  # Additional wait to ensure comments section loads

    # Scroll and collect comments
    comments = scroll_and_collect_comments(driver, video_url)

    return comments

# Function to save comments to XLSX with formatting
def save_comments_to_xlsx(video_url, comments, worksheet):
    # Write video URL in the first cell
    worksheet.cell(row=worksheet.max_row + 1, column=1).value = video_url
    worksheet.cell(row=worksheet.max_row, column=1).font = Font(bold=True)  # Highlight the header
    worksheet.cell(row=worksheet.max_row, column=1).alignment = Alignment(horizontal='center')

    # Write comments
    for comment in comments:
        worksheet.cell(row=worksheet.max_row + 1, column=2).value = comment
        worksheet.cell(row=worksheet.max_row, column=2).alignment = Alignment(wrap_text=True)

    worksheet.cell(row=worksheet.max_row + 1, column=1).fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
    worksheet.cell(row=worksheet.max_row, column=2).fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")

# Prompt for YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=Pa-74xrN3wc",
    "https://www.youtube.com/watch?v=sk1i_NcKlWY",
]

# Create a new workbook
wb = Workbook()
ws = wb.active

# Set header row
ws['A1'] = 'Video URL'
ws['B1'] = 'Comment'

# Apply formatting
header_font = Font(bold=True)
header_fill = PatternFill(start_color="A6A6A6", end_color="A6A6A6", fill_type="solid")
for cell in ws["1:1"]:
    cell.font = header_font
    cell.fill = header_fill
ws.freeze_panes = 'A2'

# Iterate through video URLs
for video_url in video_urls:
    if video_url:
        comments = get_youtube_comments(video_url)
        save_comments_to_xlsx(video_url, comments, ws)

# Auto-adjust column width
for column in ws.columns:
    max_length = 0
    for cell in column:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2) * 1.2
    ws.column_dimensions[column[0].column_letter].width = adjusted_width

# Save the workbook
wb.save('yt_comments.xlsx')


files.download('yt_comments.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>